<a href="https://colab.research.google.com/github/YuvrajPaarth/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets


In [ ]:
import opendatasets as od
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from sklearn.linear_model import LinearRegression


In [ ]:
file = od.download("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: yuvrajpaarth
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


100%|██████████| 25.7M/25.7M [00:00<00:00, 87.6MB/s]


In [ ]:
file = od.download("https://www.kaggle.com/datasets/virenmohanlal/word-indexes")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: yuvrajpaarth
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/virenmohanlal/word-indexes


100%|██████████| 643k/643k [00:00<00:00, 66.6MB/s]

In [ ]:
file = pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
file.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming the dataset contains a column named 'review'
imdb_reviews, test_reviews = train_test_split(
    file, test_size=0.2, random_state=42)  # 80% train, 20% test

# Display the counts
print("Number of IMDb Reviews (Train):", len(imdb_reviews))
print("Number of Test Reviews:", len(test_reviews))

Number of IMDb Reviews (Train): 40000
Number of Test Reviews: 10000


In [ ]:
imdb_reviews.head()

,review,sentiment
39087,That's what I kept asking myself during the ma...,negative
30893,I did not watch the entire movie. I could not ...,negative
45278,A touching love story reminiscent of In the M...,positive
16398,This latter-day Fulci schlocker is a totally a...,negative
13653,"First of all, I firmly believe that Norwegian ...",negative


In [ ]:
word_indexes = pd.read_csv('/content/word-indexes/word_indexes.csv')
word_indexes.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


In [ ]:
word_indexes = dict(zip(word_indexes.Words, word_indexes.Indexes))

In [ ]:
def review_encoder(text):
  arr = []
  for word in text:
    if word in word_indexes:
      arr.append(word_indexes[word])
    else:
      arr.append(0)

  return arr

In [ ]:
train_data, train_label = imdb_reviews['review'], imdb_reviews['sentiment']
test_data, test_label = test_reviews['review'], test_reviews['sentiment']

In [ ]:
train_data = train_data.apply(lambda review: review.split())
test_data = test_data.apply(lambda review: review.split())

In [ ]:
train_data = train_data.apply(review_encoder)
test_data = test_data.apply(review_encoder)

In [ ]:
def encode_sentiments(sentiment):
  if sentiment == 'positive':
    return 1
  else:
    return 0

train_label = train_label.apply(encode_sentiments)
test_label = test_label.apply(encode_sentiments)

In [ ]:
padding_value = word_indexes.get('<PAD>', 0)  # Get index of '<PAD>', default to 0 if not found

train_data=keras.preprocessing.sequence.pad_sequences(train_data, value=padding_value, padding='post', maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data, value=padding_value, padding='post', maxlen=500)

Word Embedding


In [ ]:
# Get the maximum word index from your training and test data
max_word_index_train = max(max(sequence) for sequence in train_data)
max_word_index_test = max(max(sequence) for sequence in test_data)
max_word_index = max(max_word_index_train, max_word_index_test)


# Increase the vocabulary size of the Embedding layer
model = keras.Sequential([
    keras.layers.Embedding(max_word_index + 1, 16, input_length=500),  # +1 to include the max index
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, train_label, epochs=30, batch_size=512, validation_data=(test_data, test_label))

In [ ]:
loss,accuracy=model.evaluate(test_data, test_label)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8719 - loss: 0.2988


In [ ]:
test_reviews.tail()

,review,sentiment
28567,Although Casper van Dien and Michael Rooker ar...,negative
25079,I liked this movie. I wasn't really sure what ...,positive
18707,Yes non-Singaporean's can't see what's the big...,positive
15200,"As far as films go, this is likable enough. En...",negative
5857,I saw Anatomy years ago -- dubbed at a friends...,positive


In [ ]:
index = np.random.randint(1,1000)

In [ ]:
user_review = test_reviews.iloc[index]
print(user_review)

review       THE ODD COUPLE (3+ outta 5 stars)<br /><br />L...
sentiment                                             positive
Name: 15769, dtype: object


In [ ]:
user_review = test_data[index]
user_review = np.array([user_review])
if (model.predict(user_review) > 0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
positive sentiment
